In [ ]:
%pip install -r requirement.txt

In [2]:
import numpy as np
import pickle

In [3]:
class Board:
    def __init__(self):
        """
        Menginisialisasi objek Board untuk permainan Tic-Tac-Toe.
        Board direpresentasikan sebagai array NumPy 1D berukuran 9.
        """
        self.board = np.zeros(9, dtype=int)  # Board 1D: 0=kosong, 1=Pemain 1, -1=Pemain -1
        self.player = 1  # Pemain saat ini: 1 atau -1
        self.winner = 0  # Pemenang: 0=belum ada, 1=Pemain 1, -1=Pemain -1, 2=Seri
        self.history = []  # Riwayat langkah-langkah board

    def reset(self):
        """
        Mengatur ulang board ke keadaan awal untuk permainan baru.
        """
        self.board = np.zeros(9, dtype=int)
        self.player = 1
        self.winner = 0
        self.history = []

    def available_moves(self):
        """
        Mengembalikan daftar indeks langkah yang tersedia (sel yang kosong).
        """
        return [i for i, x in enumerate(self.board) if x == 0]

    def make_move(self, index):
        """
        Melakukan langkah pada indeks yang diberikan.
        Mengembalikan True jika langkah valid, False jika tidak.
        """
        if 0 <= index < 9 and self.board[index] == 0:
            self.board[index] = self.player
            self.history.append(tuple(self.board)) # Menyimpan keadaan board ke riwayat
            self.player *= -1
            self.check_winner()
            return True
        return False

    def check_winner(self):
        """
        Memeriksa apakah ada pemenang atau permainan seri.
        Memperbarui atribut self.winner.
        """
        winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8], # Rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8], # Columns
            [0, 4, 8], [2, 4, 6]             # Diagonals
        ]

        for combo in winning_combinations:
            s = self.board[combo[0]] + self.board[combo[1]] + self.board[combo[2]]
            if abs(s) == 3:
                self.winner = s // 3
                return

        if len(self.available_moves()) == 0 and self.winner == 0:
            self.winner = 2

    def get_board_state(self):
        """
        Mengembalikan keadaan board saat ini sebagai tuple (immutable).
        """
        return tuple(self.board)

    def get_player_turn(self):
        """
        Mengembalikan giliran pemain saat ini (1 atau -1).
        """
        return self.player


In [ ]:
class Agent: # Nama kelas diubah menjadi QAgent untuk membedakan
    def __init__(self, name, player_id, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay_rate=0.9999, alpha=0.9, gamma=0.9):
        """
        Menginisialisasi objek QAgent untuk bermain Tic-Tac-Toe menggunakan Tabular Q-Learning.
        
        Args:
            name (str): Nama agen.
            player_id (int): ID pemain yang diwakili agen (1 atau -1).
            epsilon_start (float): Nilai epsilon awal untuk eksplorasi.
            epsilon_end (float): Nilai epsilon minimum.
            epsilon_decay_rate (float): Tingkat peluruhan epsilon per episode.
            alpha (float): Tingkat pembelajaran (learning rate).
            gamma (float): Faktor diskon untuk nilai masa depan.
        """
        self.name = name
        self.player_id = player_id
        self.epsilon = epsilon_start
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay_rate = epsilon_decay_rate
        self.alpha = alpha
        self.gamma = gamma
        self.q_table = {}  # Kamus untuk menyimpan Q-value: {state_tuple: {action_index: Q_value}}
        self.history_state_actions = [] # Riwayat (state, action) yang diambil agen dalam satu episode

    def get_q_value(self, state, action):
        """Mengambil Q-value untuk state-action pair."""
        if state not in self.q_table:
            self.q_table[state] = {i: 0.0 for i in range(9)} # Inisialisasi Q-value untuk semua aksi menjadi 0.0
        return self.q_table[state].get(action, 0.0) # Mengembalikan 0.0 jika aksi belum ada

    def choose_action(self, board_obj):
        """
        Memilih langkah berdasarkan strategi epsilon-greedy menggunakan Q-table.
        
        Args:
            board_obj (Board): Objek Board permainan saat ini.
            
        Returns:
            int: Indeks langkah yang dipilih.
        """
        current_state = board_obj.get_board_state()
        available_moves = board_obj.available_moves()
        
        if not available_moves:
            return None

        # Eksplorasi: Pilih langkah acak dengan probabilitas epsilon
        if np.random.uniform(0, 1) <= self.epsilon:
            action = np.random.choice(available_moves)
        else:
            # Eksploitasi: Pilih langkah dengan nilai Q-value tertinggi
            q_values_for_moves = {}
            for move_index in available_moves:
                q_values_for_moves[move_index] = self.get_q_value(current_state, move_index)
            
            # Cari Q-value maksimum
            max_q_value = -float('inf')
            for q_val in q_values_for_moves.values():
                if q_val > max_q_value:
                    max_q_value = q_val
            
            # Kumpulkan semua aksi yang memiliki Q-value maksimum
            best_actions = [move for move, q_val in q_values_for_moves.items() if q_val == max_q_value]
            
            # Pilih satu aksi secara acak dari yang terbaik (untuk tie-breaking)
            action = np.random.choice(best_actions)
        
        return action

    def add_state_action(self, state, action):
        """
        Menambahkan pasangan (state, action) yang diambil agen ke riwayat.
        """
        self.history_state_actions.append((state, action))

    def learn(self, board_obj):
        """
        Memperbarui nilai Q-value berdasarkan hasil permainan (reward) menggunakan Q-learning.
        """
        reward = 0
        if board_obj.winner == self.player_id:
            reward = 1      # Agen menang
        elif board_obj.winner == -self.player_id:
            reward = -1     # Agen kalah
        elif board_obj.winner == 2:
            reward = 0.5    # Permainan seri

        # Iterasi mundur melalui riwayat (state, action)
        # Dengan Q-Learning, kita memperbarui Q(S,A) berdasarkan Q(S',A')
        # S_t+1 adalah state setelah aksi A_t, A_t+1 adalah aksi optimal di S_t+1
        
        # Q-learning off-policy: Q(S,A) <- Q(S,A) + alpha * [R + gamma * max_a' Q(S',a') - Q(S,A)]
        
        # Jika ini adalah state terminal, reward langsung diberikan, tidak ada Q(S',A')
        
        # Dapatkan state_action terakhir dari history
        # Perhatikan bahwa history_state_actions sudah dalam urutan kronologis.
        # Kita akan iterasi dari yang terakhir (state sebelum terminal) ke yang pertama.

        # Nilai Q max untuk state terminal adalah 0 (karena tidak ada aksi lagi)
        next_q_max = 0.0

        # Iterasi mundur melalui riwayat (state, action)
        for state, action in reversed(self.history_state_actions):
            # Pastikan state ada di q_table
            if state not in self.q_table:
                self.q_table[state] = {i: 0.0 for i in range(9)}

            # Hitung TD Target
            if next_q_max is None: # Ini hanya akan terjadi untuk state terminal
                td_target = reward
            else:
                td_target = reward + self.gamma * next_q_max
            
            # Hitung TD Error
            td_error = td_target - self.get_q_value(state, action)
            
            # Perbarui Q-value
            self.q_table[state][action] = self.get_q_value(state, action) + self.alpha * td_error
            
            # Untuk iterasi berikutnya, next_q_max adalah Q-value dari state saat ini yang baru diperbarui
            # Namun, ini adalah Q-value dari state-action PAIR yang baru saja diambil.
            # Q-learning menggunakan max_a' Q(S',a')
            # Jadi, next_q_max untuk langkah sebelumnya adalah Q-value tertinggi dari state 'state' saat ini.
            
            # Jika state saat ini bukan state awal permainan dan bukan state terminal
            # Kita perlu mencari max Q(S,a) dari state 'state' untuk semua aksi yang mungkin.
            # Ini adalah estimasi terbaik dari nilai state 'state' itu sendiri.
            next_q_max = max(self.q_table[state].values()) # Q-value optimal dari state 'state'

            # Set reward untuk langkah selanjutnya menjadi 0 (reward hanya di akhir episode)
            reward = 0 
            
        self.history_state_actions = [] # Mengosongkan riwayat setelah pembelajaran

    def update_epsilon(self, episode):
        """
        Memperbarui nilai epsilon (untuk eksplorasi) berdasarkan episode.
        """
        self.epsilon = max(self.epsilon_end, self.epsilon_start * (self.epsilon_decay_rate ** episode))


In [ ]:

filename = 'tictactoe_agent_x.pkl'
filename2 = 'tictactoe_agent_o.pkl'

try:
    with open(filename, 'rb') as f:
        agent_x = pickle.load(f) # Load agent
    print(f"Agen AI berhasil diload ke '{filename}'")
except Exception as e:
    print(f"Terjadi kesalahan saat Load agen: {e}")
    agent_x = Agent(name="Q-Agent X", player_id=1, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay_rate=0.99995, alpha=0.1, gamma=0.9)
try:
    with open(filename2, 'rb') as f:
        agent_o = pickle.load(f) # Load agent_x
    print(f"Agen AI berhasil diload ke '{filename2}'")
except Exception as e:
    print(f"Terjadi kesalahan saat Load agen: {e}")
    agent_o = Agent(name="Q-Agent O", player_id=-1, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay_rate=0.99995, alpha=0.1, gamma=0.9)

# Inisialisasi agen
# Epsilon decay rate disesuaikan sedikit untuk eksplorasi lebih lama

# Inisialisasi board permainan
game_board = Board()

num_episodes = 50000 # Jumlah episode pelatihan yang ditingkatkan

for episode in range(num_episodes):
    game_board.reset() # Reset board untuk setiap episode baru
    current_agent = None

    # Loop permainan sampai ada pemenang atau seri
    while game_board.winner == 0 and len(game_board.available_moves()) > 0:
        current_player_id = game_board.get_player_turn()
        
        if current_player_id == 1:
            current_agent = agent_x
        else:
            current_agent = agent_o
        
        current_state = game_board.get_board_state()
        action = current_agent.choose_action(game_board)

        if action is not None:
            current_agent.add_state_action(current_state, action) # Tambahkan (state, action) ke riwayat agen
            game_board.make_move(action) # Lakukan langkah di board
        else:
            # Ini bisa terjadi jika tidak ada langkah yang tersedia (permainan sudah selesai)
            # atau jika ada bug yang menyebabkan agen memilih None.
            break 
    
    # Setelah permainan selesai, agen belajar dari hasilnya
    agent_x.learn(game_board)
    agent_o.learn(game_board)

    # Update epsilon untuk setiap agen
    agent_x.update_epsilon(episode)
    agent_o.update_epsilon(episode)

    if (episode + 1) % 5000 == 0:
        print(f"Episode {episode + 1}/{num_episodes} selesai. Epsilon Agen X: {agent_x.epsilon:.4f}, Epsilon Agen O: {agent_o.epsilon:.4f}")

print("\n--- Pelatihan Selesai ---")
print(f"Jumlah state yang dipelajari Agen X: {len(agent_x.q_table)}")
print(f"Jumlah state yang dipelajari Agen O: {len(agent_o.q_table)}")

try:
    with open(filename, 'wb') as f:
        pickle.dump(agent_x, f) # Simpan agent1
    print(f"Agen AI berhasil disimpan ke '{filename}'")
    with open(filename2, 'wb') as f:
        pickle.dump(agent_o, f) # Simpan agent1
    print(f"Agen AI berhasil disimpan ke '{filename2}'")
except Exception as e:
    print(f"Terjadi kesalahan saat menyimpan agen: {e}")    
